![](http://)# Text Processing with Spacy
    In this project we go through dataset of reviews for items on the menu for a restaurant and offer suggestions on the poorly welcomed items and best items


In [42]:
import pandas as pd

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.nlp.ex1 import *


The business owner suggested to use diner reviews from the Yelp website to determine which dishes people liked and disliked. 

In [43]:
# Load in the data from JSON file
data = pd.read_json('../input/nlp-course/restaurant.json')
data.head()


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [44]:
data.iloc[0, -2]

"I used to work food service and my manager at the time recommended I try Defalco's. He knows food well so I was excited to try one of his favorites spots.\n\nThis place is really, really good. Lot of authentic Italian choices and they even have a grocery section with tons of legit Italian goodies. I had a Chicken Parmigiana sandwich that was to die for. Anytime my ex-manager comes back to town (he left for Vegas and I think he misses Defalco's more than anything else in the valley), he is sure to stop by and grab his favorite grub.\n\nParking is a bit tricky during busy hours and the wait times for food can get a bit long, so I recommend calling your order ahead of time (unless you want to take a look around while you wait, first-timers)."

In [45]:
data.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [47]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

In [50]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> You could group reviews by what menu items they mention, and then calculate the average rating
    for reviews that mentioned each item. You can tell which foods are mentioned in reviews with low scores,
    so the restaurant can fix the recipe or remove those foods from the menu.

In [ ]:
#Approach : We are group reviews by what menu items they mention, and then calculate the average rating for reviews
# that mentioned each item. 

In [49]:
import spacy
from spacy.matcher import PhraseMatcher

index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

# Load the SpaCy model
nlp = spacy.blank('en')

# Create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

# Add the item patterns to the matcher. 
patterns = [nlp(text) for text in menu]
# Look at https://spacy.io/api/phrasematcher#add in the docs for help with this step
# Then uncomment the lines below 

# 
matcher.add("MENU",            # Just a name for the set of rules we're matching to
           None,              # Special actions to take on matched words
           *patterns  
          )

# Find matches in the review_doc
matches = matcher(review_doc)


In [51]:
for match in matches:
   print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")

Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


#Matching on the whole dataset


In [52]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
   
    matches = matcher(doc)
    
    # Create a set of the items found in the review text
    found_items = set([doc[match[1]:match[2]]for match in matches])
#     found_items = [x.lower() for x in found_items]
    # Update item_ratings with rating for each item in found_items
    
    # Transform the item strings to lowercase to make it case insensitive
    for item_found in found_items:
        item_ratings[str(item_found).lower()].append(review.stars)



In [54]:
# Calculate the mean ratings for each menu item as a dictionary
dict = {}
for k, v in item_ratings.items():
    sum = 0
    for elem in v:
        sum += elem
    avg = sum / len(v)
    dict[k] = avg
mean_ratings = dict
# {item: sum(ratings)/len(ratings) for item, ratings in item_ratings.items()}
# Find the worst item, and write it as a string in worst_text. This can be multiple lines of code if you want.
# worst_item = ____
# min = 9000
# key = ''
# for (x, y) in dict.items():
#     if min > y:
#         min = y
#         key = x
# worst_item = listofTuples(0)

# Create a list of tuples sorted by index 1 i.e. value field     
listofTuples = sorted(mean_ratings, key=mean_ratings.get)[0]
worst_item = listofTuples
    

In [56]:
print(worst_item)
print(mean_ratings[item])

chicken cutlet
5.0


In [57]:
# Most rated items

In [58]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

                    pizza  358
                    pasta  255
                 meatball  163
              cheesesteak  146
                  calzone  110
                 eggplant   95
                  cannoli   89
             cheese steak   88
                  lasagna   83
                  purista   67
               prosciutto   63
             chicken parm   58
          italian sausage   57
             garlic bread   46
                  gnocchi   45
                spaghetti   41
                 calzones   38
                   pizzas   33
                   salami   32
            chicken pesto   30
             italian beef   29
                 tiramisu   27
                     ziti   26
            italian combo   22
         chicken parmesan   21
       chicken parmigiana   18
           mac and cheese   18
               portobello   18
                 pastrami   16
           chicken cutlet   11
         steak and cheese    9
               roast beef    7
       f

10 best and 10 worst rated items. 

In [59]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Worst rated menu items:
chicken cutlet       Ave rating: 3.55 	count: 11
turkey sandwich      Ave rating: 3.80 	count: 5
spaghetti            Ave rating: 3.85 	count: 41
italian combo        Ave rating: 3.91 	count: 22
eggplant             Ave rating: 3.97 	count: 95
italian beef         Ave rating: 4.00 	count: 29
tuna salad           Ave rating: 4.00 	count: 5
garlic bread         Ave rating: 4.02 	count: 46
meatball             Ave rating: 4.08 	count: 163
portobello           Ave rating: 4.11 	count: 18


Best rated menu items:
prosciutto           Ave rating: 4.62 	count: 63
purista              Ave rating: 4.64 	count: 67
chicken salad        Ave rating: 4.67 	count: 6
pastrami             Ave rating: 4.69 	count: 16
reuben               Ave rating: 4.80 	count: 5
steak and cheese     Ave rating: 4.89 	count: 9
artichoke salad      Ave rating: 5.00 	count: 5
fettuccini alfredo   Ave rating: 5.00 	count: 6
turkey breast        Ave rating: 5.00 	count: 1
corned beef          Ave ra